In [8]:
from flask import jsonify
from flask import Flask, url_for
from flask import request
from bigquery import get_client
import requests
import json
#import traceback

print( __name__ == '__main__') 


True


In [2]:
from flask import jsonify
from flask import Flask, url_for
from flask import request
from bigquery import get_client
import requests
import json
#import traceback

app = Flask(__name__)

@app.route('/placerid/<id>')
def api_root(id):
    try:
        val = int(id)
        json_key = 'clients_secrets.json'
        client = get_client(json_key_file=json_key, readonly=True)
        try:
            print("started querying")
            queryStatement = "select * from latlong.zillow where parcelid =" + id + ";"
            print(queryStatement) 
            job_id, _results = client.query(queryStatement,timeout = 15)
        except:
            print("Error executing query") 
        complete, row_count = client.check_job(job_id)
        if complete:
            results = client.get_query_rows(job_id)
        else:
            results = "Long time to process request"
    except ValueError:
        results = "Invalid input"
    
    
    resp = jsonify({'result': results})
    return resp

@app.route('/')
def api_articles():
    return "Server is running"

# @app.route('/articles/<float:articleid>')
# def api_article(articleid):
#     print(articleid)
#     return  "HI"

@app.route('/getNearest', methods=["POST"])
def getNearest():
    print(request.headers['Content-Type'])
    if request.method == "POST":
        try:
            json_dict = request.get_json()
            

            print(json_dict)
            exactlatvalue = json_dict['lat']
            exactlongvalue = json_dict['long']
            a1 = "SELECT parcelid,latitude, longitude, SQRT( POW(69.1 * (ifNULL(latitude,0)/1000000 - "
            a2 = "), 2) +POW(69.1 * ("
            a3 = "- ifNULL(longitude,0)/1000000) * COS(ifNULL(latitude,0)/ 57300000), 2)) AS distance"
            a4 = " FROM latlong.zillow ORDER BY distance limit 10;"
            q = a1+exactlatvalue+a2+exactlongvalue+a3+a4
            json_key = 'clients_secrets.json'
            client = get_client(json_key_file=json_key, readonly=True)
            print("started querying")
            job_id, _results = client.query(q,timeout = 25)
            complete, row_count = client.check_job(job_id)
            if complete:

                results = client.get_query_rows(job_id)
            else:
                results = "Long time to process request"
            return jsonify(results)    

        except:
            print("Error executing query") 
#            traceback.print_exc()
            return jsonify("Error executing the query")
        
        
    else:

        return """<html><body>
        Something went wrong
        </body></html>"""


# SELECT parcelid,latitude, longitude, SQRT(
#     POW(69.1 * (ifNULL(latitude,0)/1000000 -34.289776 ), 2) +
#     POW(69.1 * (-118.432085 - ifNULL(longitude,0)/1000000) * COS(ifNULL(latitude,0)/ 57300000), 2)) AS distance
# FROM latlong.zillow ORDER BY distance limit 10;


app.run()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [09/Jul/2017 13:24:01] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2017 13:24:02] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [09/Jul/2017 13:24:34] "POST /placerid/ere HTTP/1.1" 405 -


started querying
select * from latlong.zillow where parcelid =122;


127.0.0.1 - - [09/Jul/2017 13:24:54] "GET /placerid/122 HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2017 13:25:08] "GET /placerid/sdsd HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2017 13:25:32] "POST /placerid/11116947 HTTP/1.1" 405 -


started querying
select * from latlong.zillow where parcelid =11116947;


127.0.0.1 - - [09/Jul/2017 13:25:40] "GET /placerid/11116947 HTTP/1.1" 200 -


application/json
{'lat': '34.465048', 'long': '-118.568166'}
started querying


127.0.0.1 - - [09/Jul/2017 13:28:22] "POST /getNearest HTTP/1.1" 200 -
